In [0]:
'''CODE FOR ACCESING FILES FROM GOOGLE DRIVE IN COLAB'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
'''MAKING DRIVE DIRECTORY'''
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import random

In [0]:
#path = os.path.abspath('.cnn.py') #absolute path of program
#path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
#dirs = os.listdir(path+'UCMerced_LandUse/Images/') #list directory in Land Use Images folder
dirs = os.listdir('drive/Datasets/UCMerced_LandUse/Images/')
label = 0

#print(dirs)

x_test=[]
y_test=[]
x_train=[]
y_train=[]



In [0]:
for i in dirs: #loop all directory
	count = 0
	for pic in glob.glob('drive/Datasets/UCMerced_LandUse/Images/'+i+'/*.tif'): #loop all picture in directory
		im = Image.open(pic) #open image
		im = np.array(im)
		if(im.shape[0]==256) and (im.shape[1] ==256) and count <90 : #get only 90 data with image shape only 256x256
			'''r=im[:,:,0]
			g=im[:,:,1]
			b=im[:,:,2]'''
			if(count<5): # 5 data in beginning set as test data
				x_test.append(im) #append image into x_test
				y_test.append([label]) #append label in y_test
			else: #after obtaining 5 data for testing, remaining data will be used as training
				x_train.append(im) #append image into x_train
				y_train.append([label]) #append label in y_train
			count = count + 1 #count image
	label = label + 1 #after finishing in the first folder, label will be incremented 0,1..,20
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test) #setting x_train,y_train,x_test,y_test as numpy array




x_train = x_train.astype('float32') #set x_train data type as float32
x_test = x_test.astype('float32') #set x_test data type as float32

x_train /= 255 #change x_train value between 0 - 1
x_test /= 255 #change x_test value between 0 - 1

y_train = keras.utils.to_categorical(y_train, 21) #change label to binary / categorical: [1 0 0 0] = 0, [0 1 0 0] = 1, so on
y_test = keras.utils.to_categorical(y_test, 21) #change label to binary / categorical

x_train=np.reshape(x_train,(1785,256,256,3))
x_test=np.reshape(x_test,(105,256,256,3))
x_train=preprocess_input(x_train)
x_test=preprocess_input(x_test)



In [0]:
vgg_conv=VGG16(weights='imagenet',include_top=False,input_shape=(256,256,3))

In [0]:
'''
FINE TUNING: FREEZING FIRST 6 LAYERS AND FINE TUNING THE REMAINING LAYERS.

for layer in vgg_conv.layers[:6]:
  layer.trainable=False
for layer in vgg_conv.layers:
  print(layer,layer.trainable)
'''
featues_train=vgg_conv.predict(x_train)
featues_test=vgg_conv.predict(x_test)

In [0]:
print(featues_train.shape)
print(featues_test.shape)

In [0]:
'''
FROM SCRATCH

model = Sequential() #model = sequential 
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(256,256,3)))#layer convolutional 2D
model.add(MaxPooling2D(pool_size=(2,2))) #max pooling with stride (2,2)
model.add(Conv2D(32, (3, 3), activation='relu')) #layer convolutional 2D
model.add(MaxPooling2D(pool_size=(2,2))) #max pooling with stride (2,2)
model.add(Dropout(0.25)) #delete neuron randomly while training and remain 75%
model.add(Flatten()) #make layer flatten
model.add(Dense(128, activation='relu')) #fully connected layer
model.add(Dropout(0.5)) #delete neuron randomly and remain 50%
model.add(Dense(21, activation='softmax')) #softmax works


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) #setting loss function and optimizer
model.fit(x_train, y_train, batch_size=50, epochs=10, verbose=1, validation_data=(x_test, y_test)) #training with epochs 100, batch size = 50
model.save("UCMerced_LandUse_Model.h5")
#model.load_weights("UCMerced_LandUse_Model.h5")
loss, acc = model.evaluate(x_test, y_test, verbose=0) #evaluate testing data and calculate loss and accuracy
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))
'''


VGGNET AS FEATURE EXTRACTOR

X_train=np.reshape(featues_train,(1785,32768))
X_test=np.reshape(featues_test,(105,32768))
model=Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(1024,input_dim=32768, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512,input_dim=1024,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,input_dim=512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.summary()

In [0]:
model=load_model("UCMerced_LandUse_Model.h5")
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=150, verbose=1, validation_data=(X_test, y_test))
model.save("UCMerced_LandUse_Model.h5")
loss, acc = model.evaluate(X_test, y_test, verbose=0) #evaluate testing data and calculate loss and accuracy
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [0]:
'''TESTING RANDOM IMAGE'''
i=random.randint(1,104)
img=x_test[i]
test_img=np.reshape(img,(1,256,256,3))
img_class=model.predict_classes(test_img)
prediction=img_class[0]
print("Prediction:",prediction)
print("Class Label:",y_test[i])
plt.imshow(img)
plt.show()
